# CMOR 461 / INDE 561 — Homework 8

## Problem 1: B&B Bookstore Inventory Optimization

**Scenario:**  
B&B, a local bookstore, must decide how many copies of a new book to order. They can only place one order before the selling season, which lasts one year. Afterward, unsold books are scrapped. 

### Given:

- **Retail price:** $20  
- **Wholesale price (from publisher):** $11  
- **Publisher's raw-material cost:** $3.75  
- **Bookstore shipping & handling cost:** $1.20  
- **Demand:** Normally distributed  
  - Mean: 1200  
  - Standard deviation: 340  
- **Unsold book scrap value:** $0.65  
- **Loss-of-goodwill cost for unmet demand:**  
  - Bookstore: $9.00  
  - Publisher: $4.00  


### Questions:

---

**a)** What is the bookstore’s optimal order quantity?

---

Per the derivation provided in the Canvas files.

We want

$$
\frac{r-w}{r-v}=F(Q^*_r)
$$

Meaning the CDF of the demand must be equal to the `Crtical Ratio`

In [51]:
r = 20
w = 11
v = .65
critical_ratio_r = (r-w) / (r-v)

from scipy.stats import norm

# Inputs
mean_demand = 1200
std_demand = 340

Q_r = norm.ppf(critical_ratio_r, loc=mean_demand, scale=std_demand)

print(f"Stock needed to satisfy {int(critical_ratio_r*100)}% of demand: {Q_r:.2f}")


Stock needed to satisfy 46% of demand: 1170.23


So it looks like the optimal order quantity is `1170.23` over time on average.

---

**b)** What are the expected profits for the bookstore and publisher using the optimal order quantity from part (a)? What is the total expected profit?

---

`Retailer:`

For order quantity $Q$, the expected profit function for the retailer is given by

$$
\pi_r(Q) = rs(Q) + vI(Q) - wQ.
$$

Where:

- $r$: retail price  
- $v$: salvage value  
- $w$: wholesale price  
- $Q$: order quantity  
- $s(Q)$: expected sales  
- $I(Q)$: expected inventory at the end of period  

In [52]:
s_Q_r=min(mean_demand, Q_r)
print(f"Given order quantity {Q_r:.2f}, expected sales: {s_Q_r:.2f}")

Given order quantity 1170.23, expected sales: 1170.23


In [53]:
I_Q_r = max(0, Q_r - mean_demand)
print(f"Given order quantity {Q_r:.2f}, expected inventory: {I_Q_r:.2f}")

Given order quantity 1170.23, expected inventory: 0.00


In [54]:
pi_r_Q_r = r*s_Q_r+v*I_Q_r-w*Q
print(f"Given order quantity {Q_r:.2f}, expected retailer profit: {pi_r_Q_r:.2f}")

Given order quantity 1170.23, expected retailer profit: 10532.09


`Supplier:`

In [55]:
c = 3.75
pi_w_Q_r = (w-c)*Q_r
print(f"Given order quantity {Q_r:.2f}, expected supplier profit: {pi_w_Q_r:.2f}")

Given order quantity 1170.23, expected supplier profit: 8484.18


`Whole Supply Chain:`

In [56]:
pi_0_Q_r = pi_r_Q_r+pi_w_Q_r
print(f"Total supply chain profit given order quantity {Q_r:.2f}: {pi_0_Q_r:.2f}")

Total supply chain profit given order quantity 1170.23: 19016.27


---

**c)** What order quantity maximizes total expected profit for the supply chain? What is the optimal total expected profit?

---

Here we want the critical ratio to be the following:

$$
F(Q_0^*)=\frac{r-c}{r-v}
$$


In [58]:
critical_ratio_0 = (r-c)/(r-v)
critical_ratio_0

Q_0 = norm.ppf(critical_ratio_0, loc=mean_demand, scale=std_demand)
print(f"Optimal order quantity to satisfy {int(critical_ratio_0*100)}% of demand: {Q_0:.2f}")

s_Q_0=min(mean_demand, Q_0)
print(f"\tExpected sales after order quantity Q_0: {s_Q_0:.2f}")

I_Q_0 = max(0, Q_0 - mean_demand)
print(f"\tExpected Inventory remaining after sales: {I_Q_0:.2f}")

pi_r_Q_0 = r*s_Q_0 + v*I_Q_0 - w*Q_0
print(f"\n\tProfit from stock level Q_0: ${pi_r_Q_0:.2f} USD")

pi_w_Q_0 = (w-c)*Q_r
print(f"\tProfit from stock level Q_0: ${pi_w_Q_0:.2f} USD")

pi_0_Q_0 = pi_r_Q_0+pi_w_Q_0
print(f"Profit from stock level Q_r with cost and revenue: ${pi_0_Q_r:.2f} USD")

Optimal order quantity to satisfy 83% of demand: 1537.83
	Expected sales after order quantity Q_0: 1200.00
	Expected Inventory remaining after sales: 337.83

	Profit from stock level Q_0: $7303.49 USD
	Profit from stock level Q_0: $8484.18 USD
Profit from stock level Q_r with cost and revenue: $19016.27 USD


---

**Buyback Contract Scenario:**  
The publisher offers a buyback credit of **$8**.

**d)** What is the bookstore’s optimal order quantity under this buyback contract? What are the expected profits for the bookstore and the publisher?

---

---

**e)** What buyback price will coordinate the supply chain? What are the resulting profits?

---

---

**Revenue Sharing Contract Scenario:**  
The bookstore keeps **60%** of its revenue, and gives **40%** to the publisher.

**f)** What wholesale price will coordinate the supply chain, and what will be the resulting profits?

---

## Problem 2: Two-Stage Ordering in a Supply Chain

**Scenario:**  
A supply chain has a single supplier and single retailer. The retailer can place **two orders**:  
1. Before demand is realized.  
2. After demand is realized (only if the first order is insufficient).  
Any unmet demand before the second order incurs a loss-of-goodwill cost `p`.

### Given:

- Demand is random with:
  - PDF: `f`
  - CDF: `F`
  - Mean: `μ`
- Selling price per item: `r`  
- Wholesale price per item (both orders): `w`  
- Salvage value per unit: `v`  
- First order production cost: `c₁`  
- Second order production cost: `c₂`  
- Assumptions:  
  - `c₁ < c₂`  
  - `v < c₁`  
  - `c₂ < w`  
- The manufacturer **produces to order** (no inventory).

### Sequence of Events:

1. Retailer places **first order** of size `Q`.
2. Order is **delivered immediately**.
3. **Demand is realized**.  
   - Demand ≤ Q: sales fulfilled immediately.  
   - Demand > Q: unmet demand is backordered.
4. Retailer places **second order** for unmet demand.
5. Order is **delivered immediately**, all unmet demand fulfilled.
6. **Unsold inventory** is salvaged.

### Questions:

---

**a)** Write the expected profit functions:
- Retailer’s profit: `π_r(Q)`  
- Supplier’s profit: `π_s(Q)`  
- Total supply chain profit: `Π(Q)`

---

---

**b)** Prove that the retailer’s optimal order quantity is strictly smaller than the supply chain’s optimal order quantity.  
*Conclusion: the supply chain is not coordinated.*

---

---

**c)** Consider a **buyback contract**:  
- Retailer pays wholesale price `w`  
- Supplier reimburses the retailer `b` per unsold unit  
- Let:  
  ```math
  w(b) = b + v + (p / (c₁ - v)) * (p + c₂ - c₁)
  ```
  Prove this contract **coordinates the supply chain**.  
  *Verify necessary and sufficient conditions.*

---

---

**d)** Briefly explain:
- Why the original supply chain was **not coordinated**.
- Why the **buyback contract** coordinates it.

---